# OpenAI tests for lambda - azure function converter

## Testing RAG over Code

In this Notebook we'll follow the instructions from the langchain website for RAG with code [here](https://python.langchain.com/docs/use_cases/question_answering/code_understanding)

### Install the necessary libraries

In [ ]:
%pip install langchain
%pip install openai
%pip install chromadb
%pip install tiktoken
%pip install python-dotenv

### Setup

Import the libraries and environment variables to gain access to the `Open API Key`

In [ ]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import Language

import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

api_key=os.environ['OPENAI_API_KEY']
base_url=os.environ['OPENAI_BASE_URL']
api_version=os.environ['OPENAI_API_VERSION']

print(base_url)

### Loading the code

We will upload all the go project files using the `langchain.document_loaders.TextLoader`. Let's define the target path with the code we want to use for RAG. 

In [ ]:
repo_path = "../go-examples"

We load the go code using `LanguageParser`, which will:

- Keep top-level functions and classes together (into a single document)
- Put remaining code into a separate document
- Retains metadata about where each split comes from

In [ ]:
# Load
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".go"],
    parser=LanguageParser(language=Language.GO, parser_threshold=500),
)
documents = loader.load()
len(documents)

### Splitting

Split the Document into chunks for embedding and vector storage.

We can use `RecursiveCharacterTextSplitter` with the language specified.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

go_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.GO, chunk_size=500, chunk_overlap=0
)
texts = go_splitter.split_documents(documents)
len(texts)

In [ ]:
print(texts[0:5])

### RetrievalQA

We need to store the documents in a way we can semantically search for their content.

The most common approach is to embed the contents of each document then store the embedding and document in a vector store.

When setting up the vectorstore retriever:

We test max marginal relevance for retrieval
And 8 documents returned

In [ ]:
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = AzureOpenAIEmbeddings(
    api_key=api_key,
    azure_endpoint=base_url, 
    api_version=api_version,
    azure_deployment="text-embedding-ada-002"
)

db = Chroma.from_documents(texts, embeddings)
retriever = db.as_retriever(
    search_type="mmr",  # Also test "similarity"
    search_kwargs={"k": 8},
)

### Chat
Test chat, just as we do for chatbots.

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationSummaryMemory

llm = AzureChatOpenAI(
    api_key=api_key,
    azure_endpoint=base_url, 
    api_version=api_version,
    model="gpt-4",
    temperature=0
)
memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

Let's start asking some questions about our code

In [ ]:
question = "What files include lambda code"
result = qa(question)
result["answer"]

In [ ]:
questions = [
    "What is the file hierarchy?",
    "What files use github.com/aws/aws-lambda-go/lambda?",
    "What one improvement do you propose to remove lambda code?",
]

for question in questions:
    result = qa(question)
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

In [ ]:
question = "name the files that use aws s3"
result = qa(question)
result["answer"]